In [2]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import tetrion.plots as plots
import tetrion.db
import tetrion.commands as cmd
import tetrion.options
import tetrion.clientcore
import seaborn as sns
from collections import defaultdict
import numpy as np
import warnings
warnings.simplefilter("ignore")
%matplotlib inline 
symbol = pd.read_csv('/local/dist/tetrion/conf/conf/symbol.table', skiprows= 1, header = 2,  delim_whitespace=True)
import redis
import json

In [3]:
def get_fills_from_redis(strat = 'capital_txo_main', acc='TAIFEX100',date=dt.date.today(),night_session=False, redishost='prod1.capital.radiant-knight.com'):
    r = redis.StrictRedis(host=redishost, port=6379,db=0)
    rk = '{}:{}'.format(acc, date.strftime('%Y%m%d'))
    rk_strat = '{}:{}'.format(strat, date.strftime('%Y%m%d')) 
#     rk = '*{}*'.format(acc)
    if night_session:
        rk = rk + 'E'
        rk_strat = rk_strat + 'E'
    if rk not in [k.decode() for k in r.keys()]:
        print('Key:{} not exists!'.format(rk))
        if rk_strat in [k.decode() for k in r.keys()]: 
            rk = rk_strat
        else:
            return None
    msgs = r.lrange(rk, 0, -1)
#     print(msgs)
    trd_df = pd.DataFrame([json.loads(m.decode()) for m in msgs])
    if trd_df.empty:
        print('Empty redis key')
        return None
    if trd_df['ts'][0] > 1e11:
        trd_df['time'] = trd_df['ts'].apply(lambda x:dt.datetime.fromtimestamp(x/1e6))
    else:
        trd_df['time'] = trd_df['ts'].apply(lambda x:dt.datetime.fromtimestamp(x))
    trd_df.set_index('time',inplace=True)

    return trd_df

In [8]:
tmf0310=get_fills_from_redis('capital_electron_tmf', date=dt.date(2025,3,4), redishost='prod1.capital.radiant-knight.com', night_session=False)

Key:TAIFEX100:20250304 not exists!


In [9]:
pd.set_option('display.max_rows',None)

In [ ]:
tmf0310.between_time('08:30:00','08:48:05')

,ts,instr,oflag,tid,px,alqortake,sz,theo,fwd,ffwd,...,gamma,theta,vol,mult,latefill,fee,qm,pfdelta,otags,seq
time,,,,,,,,,,,,,,,,,,,,,
2025-03-04 08:45:00.897534,1741049100897534,FUT_TAIFEX_TMF:202503,0,TAIFEX155#180,22244,alq,25,22243.5,22243.5,22243.5,...,0,0,0,10,False,1.24488,0,0,0x20000,1
2025-03-04 08:45:00.897996,1741049100897996,FUT_TAIFEX_TXF:202503,0,TAIFEX155#182,22245,take,-1,22250.5,22250.5,22240.0,...,0,0,0,200,False,0.48490,0,0,0x0,2
2025-03-04 08:45:00.901438,1741049100901438,FUT_TAIFEX_TMF:202503,0,TAIFEX155#199,22229,alq,25,22228.5,22228.5,22228.5,...,0,0,0,10,False,1.24458,0,0,0x20000,3
2025-03-04 08:45:00.901797,1741049100901797,FUT_TAIFEX_MXF:202503,0,TAIFEX155#204,22238,take,-2,22242.0,22242.0,22234.5,...,0,0,0,50,False,0.60476,0,0,0x0,4
2025-03-04 08:45:00.901814,1741049100901814,FUT_TAIFEX_MXF:202503,0,TAIFEX155#204,22237,take,-5,22240.5,22240.5,22234.5,...,0,0,0,50,False,0.60474,0,0,0x0,5
2025-03-04 08:45:07.579702,1741049107579702,FUT_TAIFEX_TMF:202503,0,TAIFEX155#933,22252,alq,-2,22251.5,22251.5,22251.5,...,0,0,0,10,False,1.24504,0,0,0x20000,6
2025-03-04 08:45:09.999648,1741049109999648,FUT_TAIFEX_TMF:202503,0,TAIFEX155#1229,22262,alq,1,22263.0,22263.0,22263.0,...,0,0,0,10,True,1.24524,0,0,0x20000,7
2025-03-04 08:45:09.999722,1741049109999722,FUT_TAIFEX_TMF:202503,0,TAIFEX155#1229,22262,alq,1,22263.0,22263.0,22263.0,...,0,0,0,10,True,1.24524,0,0,0x20000,8
2025-03-04 08:45:18.971140,1741049118971140,FUT_TAIFEX_TMF:202503,0,TAIFEX155#2324,22250,alq,-1,22249.5,22249.5,22249.5,...,0,0,0,10,True,1.24500,0,0,0x20000,9


In [7]:
tmf0310

,ts,instr,oflag,tid,px,alqortake,sz,theo,fwd,ffwd,...,gamma,theta,vol,mult,latefill,fee,qm,pfdelta,otags,seq
time,,,,,,,,,,,,,,,,,,,,,
2025-03-03 15:06:22.955976,1740985582955976,FUT_TAIFEX_TMF:202503,0,TAIFEX155#9091,22669,alq,2,22669.5,22669.5,22669.5,...,0,0,0,10,False,1.25338,0,0,0x20000,1
2025-03-03 15:06:24.166957,1740985584166957,FUT_TAIFEX_TMF:202503,0,TAIFEX155#9121,22670,alq,-2,22669.5,22669.5,22669.5,...,0,0,0,10,False,1.25340,0,0,0x20000,2
2025-03-03 15:06:30.695761,1740985590695761,FUT_TAIFEX_TMF:202503,0,TAIFEX155#9224,22671,alq,1,22672.0,22672.0,22672,...,0,0,0,10,False,1.25342,0,0,0x20000,3
2025-03-03 15:06:33.025081,1740985593025081,FUT_TAIFEX_TMF:202503,0,TAIFEX155#9260,22673,alq,-1,22672.0,22672.0,22672,...,0,0,0,10,False,1.25346,0,0,0x20000,4
2025-03-03 15:08:42.913060,1740985722913060,FUT_TAIFEX_TMF:202503,0,TAIFEX155#11519,22661,alq,1,22661.5,22661.5,22661.5,...,0,0,0,10,False,1.25322,0,0,0x20000,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-04 04:59:00.536606,1741035540536606,FUT_TAIFEX_TXF:202504,0,TAIFEX155#1269882,22229,take,-1,22210.5,22210.5,22175.5,...,0,0,0,200,False,0.48458,0,0,0x0,1587
2025-03-04 04:59:00.536633,1741035540536633,FUT_TAIFEX_TMF:202504,0,TAIFEX155#1269883,22225,take,20,22215.0,22215.0,22175.5,...,0,0,0,10,False,1.24450,0,0,0x0,1588
2025-03-04 04:59:01.937562,1741035541937562,FUT_TAIFEX_TXF:202504,0,TAIFEX155#1269905,22230,take,-1,22211.0,22211.0,22173,...,0,0,0,200,False,0.48460,0,0,0x0,1589
